In [ ]:
gmx_mpi pdb2gmx -f template.pdb -o template.gro -water tip3p
gmx editconf -f template.gro -o template_newbox.gro -c -d 1.0 -bt dodecahedron
# figure out what to do about -cs flag
# -cs spc216.gro
gmx solvate -cp template_newbox.gro -o template_solv.gro -p topol.top

In [2]:
# adding NaCl and KCl to the box

# am confused about the mdp file used. the file uses 1nm as the cutoff distances for WDW/coulomb interactions
# while the cutoff used in actual simulation is 1.2
gmx_mpi grompp -f mdp_files/ions.mdp -c template_solv.gro -p topol.top -o genion_NaCl_input.tpr
gmx_mpi genion -s genion_NaCl_input.tpr -o genion_NaCl_output.gro -conc 0.137 -pname NA -nname CL -p topol.top

gmx_mpi grompp -f mdp_files/ions.mdp -c genion_NaCl_output.gro -p topol.top -o genion_KCl_input.tpr
gmx_mpi genion -s genion_KCl_input.tpr -o genion_KCl_output.gro -conc 0.137 -pname NA -nname CL -p topol.top

In [4]:
# energy minimization
# same confusion as previous cell regarding mdp file, cutoff distances and other parameters different
gmx_mpi grompp -f mdp_files/em.mdp -c genion_KCl_output.gro -p topol.top -o em_input.tpr -maxwarn 1
gerun gmx_mpi mdrun -s em_input.tpr -o em.trr -x em.xtc -c em_output.gro -g em.log -e em.edr -v -maxh 10 &> em1_log

In [ ]:
# preliminary 2-ns simulation at 600 K in the NVT ensemble
# the paper doesn't mention position restraints. so no position restraints?
# also which thermostat to use?
gmx_mpi grompp -f mdp_files/nvt_600.mdp -c em_output.gro -p topol.top -o nvt_600_input.tpr -maxwarn 1
gmx_mpi mdrun -s nvt_600_input.tpr -o nvt_600.trr -x nvt_600.xtc -c nvt_600_output.gro -g nvt_600.log -e nvt.edr -v &> nvt_run.log

In [ ]:
# algorithm to extract 48 confirmations

In [ ]:
# NVT and NPT equilibration with position restraints on heavy atoms
